In [1]:
# Import pandas package and load data form https://www.kaggle.com/uciml/pima-indians-diabetes-database#diabetes.csv: 
import pandas as pd
d = pd.read_csv("E:\PythonLearning\diabetes.csv")

In [2]:
# Inspect our data: 
d.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
# Drop Pregnancies and Outcome column: 
d1 = d.drop(["Pregnancies", "Outcome"], axis = 1)

In [4]:
# Convert zeros to NA values: 
import numpy as np
d1 = d1.replace(0, np.nan)

In [5]:
# Replace NA values by mean: 
d1 = d1.fillna(d1.mean())

In [6]:
# Add Pregnancies column: 
df = d1.assign(Pregnancies = d["Pregnancies"])

In [7]:
# Write a function for normalizing 0 - 1 data: 
def normalize_01(data):
    nho_nhat = data.min()
    lon_nhat = data.max(0)
    scaled = (data - nho_nhat) / (lon_nhat - nho_nhat)
    return scaled

In [8]:
# Apply this function for our data: 
df = df[["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", 
        "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"]].apply(normalize_01)

In [9]:
# Add Outcome column: 
df = df.assign(Outcome = d["Outcome"])

In [10]:
# Write a function for training KNN model with a k selected and number of iterations is 30: 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

def knn_results (k_selected):

    my_rep = range(1, 31)
    accuracy_test = []
    
    for i in my_rep: 
        train, test = train_test_split(df, test_size = 0.3, random_state = i, stratify = df["Outcome"])

        X_train = train[train.columns[0:8]]
        y_train = train[train.columns[-1]]

        X_test = test[test.columns[0:8]]
        y_test = test[test.columns[-1]]

        X = df[df.columns[0:8]]
        Y = df["Outcome"]
        
        knn = KNeighborsClassifier(n_neighbors = k_selected)
        knn.fit(X_train, y_train)
        
        acc = knn.score(X_test, y_test)
        accuracy_test.append(acc)
        
    m = pd.DataFrame(accuracy_test, columns = ["Accuracy"])
    m = m.assign(K = np.repeat(k_selected, len(my_rep)))    
        
    return(m)     

In [11]:
# Train KNN with alternative number of neighbors: 
so_lang_gieng = np.array([1, 3, 5, 7])
total_df = pd.DataFrame()

for j in so_lang_gieng: 
    results = knn_results(k_selected = j)
    total_df = pd.concat([total_df, results])
    

In [12]:
# K optimal is 7 as we can see: 
total_df["Accuracy"].groupby(total_df["K"]).describe()

,count,mean,std,min,25%,50%,75%,max
K,,,,,,,,
1,30.0,0.697114,0.033329,0.627706,0.676407,0.699134,0.714286,0.757576
3,30.0,0.737374,0.024550,0.679654,0.722944,0.735931,0.748918,0.796537
5,30.0,0.749928,0.025208,0.705628,0.731602,0.753247,0.769481,0.805195
7,30.0,0.752958,0.023352,0.714286,0.735931,0.748918,0.766234,0.800866
